# **Farmer's Problem:**

Problem Source: Birge, John R., and Francois Louveaux. Introduction to stochastic programming. Springer Science & Business Media, 2011.

**Problem Statement:**

Consider a European farmer who specializes in raising wheat, corn, and sugar beets on his $500$ acres of land. During the winter, he wants to decide how much land to devote to each crop.

The farmer knows that at least $200\ T$ of wheat and $240\ T$ of corn are needed for cattle feed. These amounts can be raised on the farm or bought from a wholesaler. Any production in excess of the feeding requirement would be sold. Over the last decade, mean selling prices have been $170$ and $150$ per ton of wheat and corn, respectively. The purchase prices are $40\ \%$ more than this due to the wholesaler’s
margin and transportation costs.

Another profitable crop is sugar beet, which he expects to sell at $36\ T$; however,the European Commission imposes a quota on sugar beet production. Any amountin excess of the quota can be sold only at $10\ T$. The farmer’s quota for next year is $6000\ T$.

Based on past experience, the farmer knows that the mean yield on his land is roughly $2.5\ T$, $3\ T$, and $20\ T$ per acre for wheat, corn, and sugar beets, respectively.

A first possibility is to assume some correlation among the yields of the different crops. A very simplified representation of this would be to assume that years are good, fair, or bad for all crops, resulting in above average, average, or below average yields for all crops. To fix these ideas, $“above”$ and $“below”$ average indicate a yield $20\ \%$ above or below the mean yield. For simplicity, we assume that weather conditions and yields for the farmer do not have a significant impact on prices.

**Modeling:**

$\\ $

$Objective\ Function:\\ $

$Min\ 150X_{1} + 230X_{1}+ 260X_{1} \\
-\frac{1}{3} (170W_{11} - 238y_{11} + 150W_{21} - 210y_{21} + 36W_{31} + 10W_{41}) \\
-\frac{1}{3} (170W_{12} - 238y_{12} + 150W_{22} - 210y_{22} + 36W_{32} + 10W_{42}) \\
-\frac{1}{3} (170W_{13} - 238y_{13} + 150W_{23} - 210y_{23} + 36W_{33} + 10W_{43})$

$\\ $

$Constraints:\\ $

$X_{1} + X_{1} + X_{1} \leq 500$

$3X_{1} + y_{11} - W_{11} \geq 200$

$3.6X_{2} + y_{21} - W_{21} \geq 240$

$W_{31} + W_{41} \leq 24X_{3}$

$W_{31} \leq 6000$

$2.5X_{1}+ y_{12} - W_{12} \geq 200$

$3X_{2} + y_{22} - W_{22} \geq 240$

$W_{32} + W_{42} \leq 20X_{3}$

$W_{32} \leq 6000$

$2X_{1} + y_{13} - W_{13} \geq 200$

$2.4X_{2} + y_{23} - W_{23} \geq 240$

$W_{33} + W_{43} \leq 16X_{3}$

$W_{33} \leq 6000$

$X,y,W \geq 0$

In [1]:
!pip install pulp
from pulp import *
import numpy as np
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 15.8 MB/s 


In [2]:
#Defining the problem

total_acres = 500
products = ['wheat', 'corn','sugar']
costs = {'wheat':150, 'corn':230, 'sugar':260}
min_requirement = {'wheat':200, 'corn':240}
min_yields = {'wheat':2.5, 'corn':3, 'sugar':20}
scenarios = {'s1':0.8, 's2':1, 's3':1.2}
sales_price = {'wheat':170, 'corn':150, 'sugar_under':36, 'sugar_over':10}
probability = {'s1':1/3, 's2':1/3, 's3':1/3}
purchase_price = {'wheat':238, 'corn':210}

In [3]:
#Defining variables

X = LpVariable.dicts('X', ((p) for p in products), lowBound = 0, cat = 'Continuous')
y = LpVariable.dicts('y', ((p,s) for p in purchase_price for s in scenarios), lowBound = 0, cat = 'Continuous')
W = LpVariable.dicts('W', ((p,s) for p in sales_price for s in scenarios), lowBound = 0, cat = 'Continuous')

In [4]:
#Defining Objective Function

Model = LpProblem('minimize', LpMinimize)
Model += lpSum(costs[p]*X[p] for p in products) - lpSum((probability[s]*sales_price[sp]*W[sp,s] for s in scenarios for sp in sales_price))\
+ lpSum((probability[s]*purchase_price[pp]*y[pp,s]) for s in scenarios for pp in purchase_price)

In [5]:
#Constraints

Model += lpSum(X[p] for p in products) <= total_acres

for s in scenarios:
  for pp in purchase_price:
    Model += (scenarios[s]*min_yields[pp]*X[pp]) + y[pp,s] - W[pp,s] >= min_requirement[pp]

for s in scenarios:
  Model += lpSum(W[p,s] for p in ['sugar_under', 'sugar_over']) <= (scenarios[s]*min_yields['sugar']*X['sugar'])

Model += W['sugar_under',s] <= 6000

In [6]:
#Model summary

Model

minimize:
MINIMIZE
-50.0*W_('corn',_'s1') + -50.0*W_('corn',_'s2') + -50.0*W_('corn',_'s3') + -3.333333333333333*W_('sugar_over',_'s1') + -3.333333333333333*W_('sugar_over',_'s2') + -3.333333333333333*W_('sugar_over',_'s3') + -12.0*W_('sugar_under',_'s1') + -12.0*W_('sugar_under',_'s2') + -12.0*W_('sugar_under',_'s3') + -56.666666666666664*W_('wheat',_'s1') + -56.666666666666664*W_('wheat',_'s2') + -56.666666666666664*W_('wheat',_'s3') + 230*X_corn + 260*X_sugar + 150*X_wheat + 70.0*y_('corn',_'s1') + 70.0*y_('corn',_'s2') + 70.0*y_('corn',_'s3') + 79.33333333333333*y_('wheat',_'s1') + 79.33333333333333*y_('wheat',_'s2') + 79.33333333333333*y_('wheat',_'s3') + 0.0
SUBJECT TO
_C1: X_corn + X_sugar + X_wheat <= 500

_C2: - W_('wheat',_'s1') + 2 X_wheat + y_('wheat',_'s1') >= 200

_C3: - W_('corn',_'s1') + 2.4 X_corn + y_('corn',_'s1') >= 240

_C4: - W_('wheat',_'s2') + 2.5 X_wheat + y_('wheat',_'s2') >= 200

_C5: - W_('corn',_'s2') + 3 X_corn + y_('corn',_'s2') >= 240

_C6: - W_('wheat',

In [7]:
#Solving model

Model.solve()
print('Optimal Solution:', -pulp.value(Model.objective))

Optimal Solution: 108390.0


In [8]:
#Optimal values of variables

for variables in Model.variables():
  print('{}:{}'.format(variables.name, variables.varValue))

W_('corn',_'s1'):0.0
W_('corn',_'s2'):0.0
W_('corn',_'s3'):48.0
W_('sugar_over',_'s1'):0.0
W_('sugar_over',_'s2'):0.0
W_('sugar_over',_'s3'):0.0
W_('sugar_under',_'s1'):4000.0
W_('sugar_under',_'s2'):5000.0
W_('sugar_under',_'s3'):6000.0
W_('wheat',_'s1'):140.0
W_('wheat',_'s2'):225.0
W_('wheat',_'s3'):310.0
X_corn:80.0
X_sugar:250.0
X_wheat:170.0
y_('corn',_'s1'):48.0
y_('corn',_'s2'):0.0
y_('corn',_'s3'):0.0
y_('wheat',_'s1'):0.0
y_('wheat',_'s2'):0.0
y_('wheat',_'s3'):0.0


In [9]:
VNames = []
for variables in Model.variables():
  VNames.append(variables.name)

VValue = []
for variables in Model.variables():
  VValue.append(int(variables.varValue))

data = {'Variables': VNames, 'Value': VValue}
pd.DataFrame(data, index = range(1, len(VValue)+1))

,Variables,Value
1,"W_('corn',_'s1')",0
2,"W_('corn',_'s2')",0
3,"W_('corn',_'s3')",48
4,"W_('sugar_over',_'s1')",0
5,"W_('sugar_over',_'s2')",0
6,"W_('sugar_over',_'s3')",0
7,"W_('sugar_under',_'s1')",4000
8,"W_('sugar_under',_'s2')",5000
9,"W_('sugar_under',_'s3')",6000
10,"W_('wheat',_'s1')",140


In [10]:
print('Current Status: ', LpStatus[Model.status])

Current Status:  Optimal
